## Importing libraries

In [ ]:
import requests
from bs4 import BeautifulSoup
from pprint import pprint
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import datetime

## Data Scraping

In [ ]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

my_headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OSX 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8"}

def get_soup(url):
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.content, 'html.parser')
    return soup

### Category

Scrape all available category name and link from the "homepage"

In [ ]:
category_url = 'https://www.amazon.com/b/ref=glow_cls?node=17938598011'
amazon_soup = get_soup(url=category_url)

In [ ]:
# Search for all categories
att0=0
departments = []
while((departments==[]) & (att0<=1000)):
    amazon_soup = get_soup(url=category_url)
    departments = amazon_soup.find('div', {'class':"left_nav browseBox"})
    att0=att0+1

In [ ]:
categories_soup = departments.find_all('li')[2:]

In [ ]:
# Store each category's name and link to a dictionary
categories = {}

for category in categories_soup:
    name = category.get_text().strip()
    link = 'https://www.amazon.com' + category.find('a')['href']
    categories[name] = link

In [ ]:
pprint(categories)

### Sub category

Using the link in each category, scrape all subcategories' name and link

In [ ]:
"""
Expected data structure:
category_and_sub = {
                    "arts" : [["crafting", "crafting_link"],
                                ["fabric", "fabric_link"],
                                ["sewing", "sewing_link"]]
                    }
"""

In [ ]:
categories_and_subs = {}
not_scraped = {}

for category in categories:
    # Find all subcategory from the current category link
    subcategories_soup = get_soup(url=categories[category])
    subcategories = subcategories_soup.find_all('a', {'class':'a-color-base a-link-normal'})[0:]
    
    categories_and_subs[category] = []
    
    print(f"Scraping {category}")
    
    for subcategory in subcategories:
        # Store the name and link of each subcategory found in the category
        name = subcategory.get_text().strip()
        link = 'https://www.amazon.com' + subcategory['href']
        
        categories_and_subs[category].append([name, link])
        print(f'--scrape {[name, link]}')
        
    # Categories with no found subcategory
    if categories_and_subs[category] == []:
        not_scraped[category] = categories[category]
        print(f'*** Could not scrape {category}')

In [ ]:
not_scraped

In [ ]:
for category in not_scraped:
    subcategories_soup = get_soup(url=not_scraped[category])
    print(category)
    subcategories = ['']
    
    categories_and_subs[category] = []
    
    for subcategory in subcategories:
        name = ''
        
        query = (category + ' ' + name).split(' ')
        query = '+'.join(query)
        link = f"https://www.amazon.com/s?k={query}"
        
        categories_and_subs[category].append([name, link])

In [ ]:
for category in categories_and_subs:
    print(category)
    pprint(categories_and_subs[category])
    print('\n')

### Products

In [ ]:
print(f"Total categories: {len(categories_and_subs)}")

subcategories_count = sum([len(categories_and_subs[c]) for c in categories_and_subs])
print(f"Total subcategories: {subcategories_count}")

# Each initial search query returns a minimun of 24 result(products), so our dataset would have an initial lenght of:
print(f"Total products: {subcategories_count * 24}")

In [ ]:
# Things are getting quite complicated. Time to bring in the pandas library
subcategory_df = pd.DataFrame(columns=['Category', 'Sub Category', 'Link'])
subcategory_df

In [ ]:
# All values are simply stored in the dataframe
i = 0
for category in categories_and_subs:
    for sub in categories_and_subs[category]:
        subcategory_df.loc[i] = [category, sub[0], sub[1]]
        i += 1

In [ ]:
assert len(subcategory_df) == subcategories_count
subcategory_df

In [ ]:
subcategory_df = subcategory_df.drop_duplicates(subset="Link", keep="first")
subcategory_df

In [ ]:
products_df = pd.DataFrame(columns=['Category', 'Sub Category', 'Product', 'Price', 'Stars_and_Count', "Brand", "Best Sellers Rank", "Release Date","Manufacturer"])
products_df

In [ ]:
i = 0
# Loop through all subcategory
for _, row in subcategory_df.iterrows():
    
    curr_subcategory = row['Sub Category']
    curr_category = row['Category']
    # For manual confirmation
    print(f"Searching {curr_subcategory}")
    print(f"Searching {curr_category}")
    print(row['Link'])
    
    # Select all available products' element
    row_soup = get_soup(url=row['Link'])
    att=0
    products_soup = []
    while((products_soup==[]) & (att<=1000)):
        row_soup = get_soup(url=row['Link'])
        products_soup = row_soup.select('div.a-section.a-spacing-small.puis-padding-left-small.puis-padding-right-small')
        att=att+1
    
    
    if (products_soup==[]):
        print(f"failure")

        
    # print(products_soup)
    
    # Loop through all products found
    for product_soup in products_soup:
        try:
            target_keys = {"Brand", "Best Sellers Rank", "Release Date", "Date First Available", "Manufacturer"}
            results = {key: None for key in target_keys}
        
            title = product_soup.select_one('h2.a-size-base-plus.a-spacing-none.a-color-base.a-text-normal').get_text()
            rating = product_soup.select_one('div.a-row.a-size-small').get_text()
            price = product_soup.select_one('span.a-price span').get_text()
            # print(price)
            
            # get product details
            link = product_soup.select_one('a.a-link-normal.s-line-clamp-4.s-link-style.a-text-normal')
            link = 'https://www.amazon.com' + link['href']

            # print(link)
            
            att2=0
            attr_soup = []
            while((attr_soup==[]) & (att2<=100)):
                detail_soup = get_soup(url=link)
                attr_soup = detail_soup.select('table#productDetails_detailBullets_sections1.a-keyvalue.prodDetTable')
                att2=att2+1

            # print(attr_soup)

            if attr_soup:
                table = attr_soup[0]  # Select the first matching table

                
                rows = table.find_all("tr")  # Extract all rows of the table
                # print(rows)
                
                for row in rows:
                    #print(row)
                    th = row.find('th').get_text(strip=True)  # Find <th> element
                    #print(th)
                    td = row.find('td').get_text(strip=True)  # Find <tb> element
                    #print(td)
                    if th in target_keys:  # Only process rows for specified keys
                        value = td if td else None  # Get <tb> text or set to None
                        results[th] = value
                        #print(results)         
                    #break one row
                    
        except AttributeError:
            print('error')
            break
        
        # Store values in our products dataframe
        if (results["Release Date"] == None):
            results["Release Date"] = results["Date First Available"]
        # print(results)
        products_df.loc[i] = [curr_category, curr_subcategory, title, price, rating, results["Brand"], results["Best Sellers Rank"], results["Release Date"],results["Manufacturer"]]
        i += 1
        
        print('arr')
        
        #break one product
        
    print()
    # break #one subcategory

In [ ]:
products_df

Data scraped without gathering info from secondary link

In [ ]:
products_df1 = pd.DataFrame(columns=['Category', 'Sub Category', 'Product', 'Price', 'Stars_and_Count'])
products_df1

In [ ]:
i = 0
# Loop through all subcategory
for _, row in subcategory_df.iterrows():
    
    curr_subcategory = row['Sub Category']
    curr_category = row['Category']
    # For manual confirmation
    print(f"Searching {curr_subcategory}")
    print(f"Searching {curr_category}")
    print(row['Link'])
    
    # Select all available products' element
    row_soup = get_soup(url=row['Link'])

    att=0
    products_soup = []
    while((products_soup==[]) & (att<=1000)):
        row_soup = get_soup(url=row['Link'])
        products_soup = row_soup.select('div.a-section.a-spacing-small.puis-padding-left-small.puis-padding-right-small')
        att=att+1
    
    if (products_soup==[]):
        print(f"failure")
        
    # Loop through all products found
    for product_soup in products_soup:
        try:
            title = product_soup.select_one('h2.a-size-base-plus.a-spacing-none.a-color-base.a-text-normal').get_text()
            rating = product_soup.select_one('div.a-row.a-size-small').get_text()
            price = product_soup.select_one('span.a-price span').get_text()
        except AttributeError:
            continue
        
        # Store values in our products dataframe
        products_df1.loc[i] = [curr_category, curr_subcategory, title, price, rating]
        i += 1
    print()

In [ ]:
products_df1

In [ ]:
products_df1.to_csv('data/products_original2.csv', index=False)

A fancy approach would be to generate the payment options based on the price of the product bought<br>
E.g. expensive products are more likely to be paid for with ATM cards or check, and cheaper products, with gift cards or cash 

In [ ]:
products_df['Price'] = products_df['Price'].str.strip('$').astype(float)
products_df

Quick analytics

In [ ]:
plt.figure(figsize=(15,6))
sns.scatterplot(data=products_df['Price'].value_counts())

mean_value = products_df['Price'].mean()
print(f'Mean: {mean_value}')

sum_value = products_df['Price'].sum()
print(f'Sum: {sum_value}')

max_value = products_df['Price'].max()
products_df.loc[products_df['Price']==max_value].iloc[0:1]

In [ ]:
payment_options = ['Cash', 'ATM card', 'Check', 'Gift card', 'Mobile payment']

payment_probs_1 = [0.7, 0.26, 0.002, 0.013, 0.025]
payment_probs_2 = [0.2, 0.55, 0.045, 0.005, 0.2]

assert sum(payment_probs_1) == 1
assert sum(payment_probs_2) == 1

In [ ]:
for i in range(products_df.shape[0]):
    
    # Expensive products are less likely to be paid with cash
    if products_df.loc[i, 'Price'] >= 300:
        payment_probs = payment_probs_2
    else:
        payment_probs = payment_probs_1
        
    products_df.loc[i, 'Payment_Type'] = np.random.choice(payment_options, p=payment_probs)
    

    
products_df['Payment_Type'].value_counts()

In [ ]:
for i in range(products_df.shape[0]):
    
    # Expensive products typical generate more profit
    if products_df.loc[i, 'Price'] >= 150:
        profit_percent = np.random.uniform(0.1, 0.25)
    else:
        profit_percent = np.random.uniform(0.05, 0.15)
    
    price = products_df.loc[i, 'Price']
    products_df.loc[i, 'Cost_Price'] = price - (profit_percent * price)

In [ ]:
products_df['Cost_Price'].round(decimals=2)

In [ ]:
start_date = datetime.date(2021, 1, 1)
end_date = datetime.date(2022, 1, 1)

days_between = (end_date - start_date).days

In [ ]:
for i in range(products_df.shape[0]):
    
    # Generate a random date
    random_day = np.random.randint(days_between)
    random_date = start_date + datetime.timedelta(days=random_day)
    
    products_df.loc[i, 'Order_Date'] = random_date

In [ ]:
products_df.rename(columns={'Sub Category':'Sub_Category',
                            'Branches':'Branch'}, inplace=True)

products_df.drop(columns=['Occurance', 'Rating_Count', 'Rating'], inplace=True)


# Reorder columns
products_df = products_df[['Branch', 'Category', 'Sub_Category', 'Product',
                           'Price', 'Cost_Price', 'Payment_Type', 'Order_Date']]

In [ ]:
products_df = products_df.sample(frac=1).reset_index(drop=True)
products_df.to_csv('products.csv', index=False)
products_df

In [ ]:
# from google.colab import files
# files.download('/content/products.csv')